# Project Signal, Image & Video

## Task

Given a video sequence that represents a person moving within an environment, track their movements and generate a spatio-temporal heatmap that represents the probability of occupancy of the various areas of the monitored environment over time.

### Dataset
The dataset I used for the project consists of videos from a ceiling camera in a corridor at the University of Trento, Povo 2. 

Given its size, the dataset is not included with the project. However, heatmaps calculated from those videos, for all the periods that the videos allowed to define, are included. These heatmaps can be viewed in Section 3 of the project.

### Section 1
This section defines the necessary modules, general functions to assist with output presentation, and file system navigation. The final cells in this part analyze the dataset to understand the available videos and set suitable time ranges.

### Section 2
This section introduces two methodologies for heatmap computation. The first couple of subsections detail the functions of the background subtraction method and a method that calculate the heatmap using YOLO people detections. The final part of this section enables the computation of the heatmaps from videos, given the time ranges. Executing this second part is not mandatory, as part 3 of the project can be used if heatmaps are already generated and placed in the appropriate folders.

### Section 3
The final section allows for the visualization of the generated heatmap and the computation of area occupancy. The process applied to the heatmap for calculating occupancy slightly differs based on the method used to generate the heatmap.
After selecting a time range, it is possible to view all the heatmaps available for that time range, and an average heatmap will be calculated over the entire time range. This facilitates the identification of the average occupancy during that period.

# Section 1 - Environment preparation and Dataset Inspection

## Modules

In this cell, the necessary modules are imported, and the drive is mounted if the project is run on Google Colab.

In [2]:
# Importing necessary modules
import pandas as pd
import numpy as np
import cv2 as cv2
import time
import random
import re
import os
import io
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython.display import clear_output
from datetime import datetime, timedelta

# Installing the avi-r module
!pip install avi-r
from avi_r import AVIReader

# Mounting the drive in case of running on Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    os.chdir('/content/drive/MyDrive/ColabSIV')
    os.listdir()
except:
    print('Not in Google Colab.')

# Confirmation message for module loading
print(f'Modules loaded!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 4.0 MB/s eta 0:00:0000:0100:01
Not in Google Colab.
Modules loaded!


## File System parameter

Parameters for managing the input and output during heatmap generation.

In [3]:

MT1_VIDEO_PATH = 'method1_background_subtraction/videos' # Path for saving videos processed by Method 1 (background subtraction).

MT1_HEATMAP_PATH = 'method1_background_subtraction/heatmaps' # Path for saving heatmaps generated by Method 1 (background subtraction).

MT2_VIDEO_PATH = 'method2_yolo_detections/videos' # Path for saving videos processed by Method 2 (YOLO).

MT2_HEATMAP_PATH = 'method2_yolo_detections/heatmaps' # Path for saving heatmaps generated by Method 2 (YOLO).

YOLO = 'people_detection_model' # Path to the folder containing the YOLO people detection model.

BACKGROUND_PATH = 'background/background.png' # Path to the image file used for background.


## General purpose functions

These functions provide general-purpose utilities for:

- Visualizing data, dataframes, and series of images.
- Retrieving videos and images from the file system.
- Manipulating datetime and time information.
- Analyzing the predefined dataset and available videos, with respect to defined time ranges.

### Data visualization

In [4]:
# Function to plot multiple images in a grid of a given dimension
def plot_images(images, subtitle='', titles=None, images_per_row=2):
    num_images = len(images)

    # Generate as many titles as images if titles are not provided
    if titles is None:
        titles = [f'Img {i+1}' for i in range(num_images)]
    elif len(titles) != num_images:
        raise ValueError("Number of titles must match number of images")

    num_rows = (num_images // images_per_row) + int((num_images % images_per_row) > 0)
    fig, axs = plt.subplots(num_rows, images_per_row, figsize=(5*images_per_row, 5*num_rows))
    axs = axs.flatten()

    for i in range(images_per_row * num_rows):
        ax = axs[i]
        if i < num_images:
            ax.imshow(images[i])
            ax.set_title(titles[i])
        ax.axis('off')

    fig.suptitle(subtitle, fontsize=16)
    plt.tight_layout()
    plt.show()

# Function to plot a single image
def plot_image(image, title=''):
    plt.figure()
    plt.imshow(image)
    plt.title(title)

    plt.axis('off')
    plt.show()

# Print a title
def print_title(title):
    print('\n--------------------------| ',title, ' |--------------------------\n')
    
# Print a dataframe in a custom format
def custom_print(dataframe, title = ''):
    print(f"{title}\n{tabulate(dataframe, headers='keys', tablefmt='simple_grid', floatfmt='.6f', showindex=True)}")

# Generate a list of file paths from a dataframe
def path_list(dataframe):
    return [os.path.join(dataframe.loc[i]['Root'], dataframe.loc[i]['Filename']) for i in range(len(dataframe))]

# Allow user to select an element from a given list, 0 to terminate.
def make_choice(choice_list, title = ''):
    clear_output()
    print(title)
    print(f'0 - Exit')
    for i, key in enumerate(choice_list):
        print(f'{i+1} - {key}')

    choice = int(input('Choice : '))
    while (choice < 1 and choice != 0 ) or choice > len(choice_list)+1:
        choice = int(input('Choice : '))
    
    if choice == 0:
        print(f'Exit.')
    return choice

### File System interaction

In [5]:
def load_video(videopath):
    """
    Load a video using AVIReader.
    
    Args:
        videopath (str): Path to the video file.
    
    Returns:
        AVIReader: AVIReader object representing the loaded video.
    """
    return AVIReader(videopath)

"""
Read images from a folder and return them as a dictionary.

Args:
    folder_path (str): Path to the folder containing the images.

Returns:
    dict: A dictionary where the keys are the image names (without file extensions) and the values
          are the corresponding image objects.
"""
def read_images_from_folder(folder_path):
    image_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_name = os.path.splitext(filename)[0]
            image_path = os.path.join(folder_path, filename)
            
            try:
                with Image.open(image_path) as image:
                    image_dict[image_name] = image.copy()
            except (IOError, OSError):
                print(f"Error while loading image: {filename}")
    
    return image_dict

### Time operations

Functions to assists in handling datetime/time information and divide the videos available inside the dataset based on predefined time ranges.

In [6]:
def extract_datetime(string):
    """
    Extracts date, start time, and end time from a given string.
    
    Args:
        string (str): The input string containing the date and time information.
    
    Returns:
        tuple: A tuple containing the extracted date, start time, and end time.
               Returns None if no match is found.
    """

    # Regular expression to extract date and time from the given string
    datetime_regex = r'(\d{4}-\d{2}-\d{2}) (\d{2}-\d{2}-\d{2})~(\d{2}-\d{2}-\d{2})'
    match = re.search(datetime_regex, string)
    
    if match: # Extract date, start time, and end time from the matched groups
        date = match.group(1)
        start_time = match.group(2)
        end_time = match.group(3)
        
        return date, start_time, end_time
    else:
        return '', '', ''


def split_videos_by_time(videos, predefined_ranges):
    """
    Splits the videos loaded with 'dataset_info' based on the given time ranges
    
    Args:
        videos (pd.DataFrame): DataFrame containing video information.
        predefined_ranges (List[Tuple[datetime.time, datetime.time]]): List of tuples
            representing the start time and end time of each predefined range.
    
    Returns:
        Tuple[Dict[str, pd.DataFrame], pd.DataFrame]: A tuple containing a dictionary
            of split dataframes and a dataframe containing statistics for each split.
    """
    dfs = {} 
    num_ranges = len(predefined_ranges)    
    dfs_stats = pd.DataFrame(columns=['Split','Duration','St.Time','En.Time','Tot.Frames','Video.Count'])
    
    for i, (range_start_time, range_end_time) in enumerate(predefined_ranges):
        # Creating an empty dataframe for each predefined time range
        df = pd.DataFrame(columns = videos.columns)
        
        for k in range(len(videos)):
            
            video_start_time = datetime.strptime(videos.loc[k]['St.Time'], "%H-%M-%S")
            video_end_time = datetime.strptime(videos.loc[k]['En.Time'], "%H-%M-%S")
            
            if (video_start_time >= range_start_time) & (video_end_time <= range_end_time):
                # Add video tu current dataframe if it belong
                df.loc[len(df)] = videos.loc[k]

        # Creating row for general statistics aggregating current dataframe data.
        row = [i+1, 
               timedelta(seconds=df['Frame'].sum() / df['FPS'].mean()), 
               range_start_time.time(), 
               range_end_time.time(), 
               df['Frame'].sum(),
               len(df)]

        # Update single range list of dataframe and the dataframe of statistics (dfs_stats)
        dfs_stats.loc[i] = row
        dfs.update({f'{range_start_time.time()}-{range_end_time.time()}' : df})
        
    return dfs, dfs_stats

### Video operations

All videos are read using avi-r (all available videos from the camera are in .avi format).

In [7]:
def video_info(video, videopath):
    """
    Get information about a video.
    
    Args:
        video (AVIReader): AVIReader object representing the video.
        videopath (str): Path to the video file.
    
    Returns:
        list: A list containing the video information including the filename, duration, date,
              start time, end time, width, height, frame rate, and total frames.
    """
    total_frame = video.num_frames
    if total_frame < 0:
        print(f"Error: Total frame < 0 for video {videopath}")
        return []
    info = [os.path.basename(videopath), 
            timedelta(seconds = total_frame / video.frame_rate),
            *extract_datetime(videopath), 
            video.width, video.height, video.frame_rate, total_frame]
    
    return info


def dataset_info(dataset_folder):
    """
    Get information about the dataset.

    Args:
        dataset_folder (str): Path to the dataset folder.

    Returns:
        tuple: A tuple containing a dataframe with the dataset statistics and a list of file paths
               corresponding to the videos in the dataset.
    """
    stats = pd.DataFrame(columns=['Root','Filename','Duration', 'Date', 'St.Time', 'En.Time','Width', 'Height', 'FPS', 'Frame'])
    for root, _, files in os.walk(dataset_folder):
        for filename in files:
            if os.path.splitext(filename)[1].lower() in ['.mp4', '.avi', '.mkv']:
                video = load_video(os.path.join(root, filename))
                info = video_info(video, filename) if video else []
                if info:
                    stats.loc[len(stats)] = [root, *info]
                video.close()
            else:
                print(f"Skipping non-video file: {filename}")
    return stats, path_list(stats)

## Dataset Analysis

This section of the inspect the dataset, to understand the characteristics of the available videos and their time distribution.

### Inspect dataset - General

Show informationa about the videos inside the dataset folder

In [8]:
# Path to folder in which videos are stored.
DATESET_PATH = 'dataset'

# Retrieve dataset information
videos_info, paths = dataset_info(DATESET_PATH)

# Show information
print(f'\n-> Available videos : {len(paths)}')
custom_print(videos_info, '\nDetails:')


-> Available videos : 0

Details:
┌────────┬────────────┬────────────┬────────┬───────────┬───────────┬─────────┬──────────┬───────┬─────────┐
│ Root   │ Filename   │ Duration   │ Date   │ St.Time   │ En.Time   │ Width   │ Height   │ FPS   │ Frame   │
├────────┼────────────┼────────────┼────────┼───────────┼───────────┼─────────┼──────────┼───────┼─────────┤
└────────┴────────────┴────────────┴────────┴───────────┴───────────┴─────────┴──────────┴───────┴─────────┘


### Time ranges

In [9]:
# Time ranges for analysis of the available videos.

time_ranges = [
    (pd.to_datetime("08:00:00", format="%H:%M:%S"), pd.to_datetime("08:29:59", format="%H:%M:%S")),
    (pd.to_datetime("08:30:00", format="%H:%M:%S"), pd.to_datetime("09:00:00", format="%H:%M:%S")),
    (pd.to_datetime("12:00:00", format="%H:%M:%S"), pd.to_datetime("12:29:59", format="%H:%M:%S")),
    (pd.to_datetime("12:30:00", format="%H:%M:%S"), pd.to_datetime("12:59:59", format="%H:%M:%S")),
    (pd.to_datetime("13:00:00", format="%H:%M:%S"), pd.to_datetime("14:00:00", format="%H:%M:%S")),
    (pd.to_datetime("16:00:00", format="%H:%M:%S"), pd.to_datetime("16:59:59", format="%H:%M:%S")),
    (pd.to_datetime("17:00:00", format="%H:%M:%S"), pd.to_datetime("17:29:59", format="%H:%M:%S")),
    (pd.to_datetime("17:30:00", format="%H:%M:%S"), pd.to_datetime("18:00:00", format="%H:%M:%S"))
]

### Inspect dataset - With time ranges

Displays information about the videos in the dataset, categorizing them based on predefined time ranges.

In [10]:
# Split videos on time ranges
splits, split_stats = split_videos_by_time(videos_info.copy(), predefined_ranges = time_ranges)

# Show splitted videos information
print(f'\nRequested time Ranges:\n')
for (start,end) in time_ranges:
    print(f'- {start.time()} - {end.time()}')
print('\nUsed Videos with given time ranges:', split_stats['Video.Count'].sum(),'/', len(videos_info), '\n')
split_stats = split_stats.rename(columns = {'St.Time':'Start time','En.Time':'End Time','Split':'Group','Tot.Frames':'Total Frames','Video.Count':'Available Videos'})
custom_print(split_stats.replace({'St.time':'Start time','En.Time':'End Time','Split':'Video'}).set_index('Group'), '\nAvailable videos for each time range')
a = split_stats['Duration'].mean()
b = split_stats['Total Frames'].mean()
print(f'Duration: {a} - {b}')
for period, df in splits.items():
    print(f'\n-----> Range: {period} ')
    custom_print(df,'Videos:')

ValueError: cannot convert float NaN to integer

# Section 2 - Computing heatmaps with two methodology

(This part can be not executed if there already heatmaps in the appropriate folders to analyze)

## Heatmap - Background subtraction

Background subtraction technique involves isolating moving objects in a video by separating them from the background. The object is identified by subtracting the current image from a background model. The background model is the image of the video without any moving objects. It's particularly effective for videos where the main subject of interest is moving and the background remains largely static.

The code uses the K-Nearest Neighbors (KNN) background subtractor model provided by OpenCV. The KNN model is a non-parametric learning algorithm that identifies moving objects based on pixel density in the image.

In summary, this method computes the heatmap considering all the movement within the image.

### Mappings for files labelling

Functions to get the string for the used parameters. These strings are used to create the names of the saved heatmaps and videos.

In [11]:
def morph_name(morph):
    """
    This function maps the OpenCV morphological operation type to its corresponding name. 
    It takes a morphological operation type as input and returns the corresponding name. 
    If the provided morphological operation type is not mapped, it returns 'Unknown'.
    """
    morph_dict = {
        cv2.MORPH_ERODE: 'MORPH_ERODE', # Default
        cv2.MORPH_DILATE: 'MORPH_DILATE',
        cv2.MORPH_OPEN: 'MORPH_OPEN',
        cv2.MORPH_CLOSE: 'MORPH_CLOSE',
    }
    return morph_dict.get(morph, 'Unknown')


def morph_shape_name(morph_shape):
    """
    This function maps the OpenCV morphological shape type to its corresponding name. 
    It takes a morphological shape type as input and returns the corresponding name. 
    If the provided morphological shape type is not mapped, it returns 'Unknown'.
    """
    morph_shape_dict = {
        cv2.MORPH_RECT: 'MORPH_RECT', # Default
        cv2.MORPH_CROSS: 'MORPH_CROSS',
        cv2.MORPH_ELLIPSE: 'MORPH_ELLIPSE',
    }
    return morph_shape_dict.get(morph_shape, 'Unknown')


### Background subtraction

The heatmap_background_subtraction() function implemented in this code utilizes background subtraction method to generate a heatmap.

In [12]:
def apply_morph(image: np.ndarray, kernel_operation=cv2.MORPH_ERODE, kernel_size: (int, int) = (3, 3), kernel_shape = cv2.MORPH_RECT, make_gaussian =  True):
    """
    Apply opencv morphological operation to image and return it.
    
    Args:
        image (np.ndarray): The source image to which the morphological operation is applied.
        morph_type: Opencv morphological operation type.
        kernel_size (int, int): Tuple of ints, representing size of kernel for morphological operation.
        make_gaussian (bool): If true apply gaussian blur, otherwise not.
    Returns:
        Source image with applied chosen morphological operation.
    
    """
    kernel = cv2.getStructuringElement(kernel_shape, kernel_size) # Matrix used to modify the shape of objects in the image during the morphological operation
    if make_gaussian:
        image = cv2.GaussianBlur(image, (3, 3), 0) # This filter is used to reduce the noise and details of the image.
    return cv2.morphologyEx(image, kernel_operation, kernel) # Essentially removing pixels at the edges of objects in the image


def add_images(image1: np.ndarray, image2: np.ndarray):
    """
    Add two images together. 
    
    Note:
        Colors values can be bigger then 255 restriction.
        Use np.uint64 cast so images are able to expand colors restriction above 255.
        In order to view this image normalize it.
    Args:
        image1 (np.ndarray): First image to add.
        image2 (np.ndarray): Second image to add.
    Returns:
        np.ndarray: Output image represeting addition of image1 and image2.
    """
    return np.array(image1, dtype=np.uint64) + np.array(image2, dtype=np.uint64)


def normalize_image(image: np.ndarray):
    """
    Normalize image to 0-255 range, so it is viewed correctly.
    
    Args:
        image (np.ndarray): Image for which normalization should be applied.
    Returns:
        np.ndarray: Normalized image.
    """
    return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)


def apply_heatmap_colors(image: np.ndarray):
    """
    Apply colors for heatmap visualisation.
    
    Args:
        image (np.ndarray): Image for which heatmap colors should be applied.
    Returns:
        np.ndarray: Image with applied heatmap colors.
    """
    return cv2.applyColorMap(image, cv2.COLORMAP_TURBO)


def superimpose(image1: np.ndarray, image2: np.ndarray, alpha: float = 0.5):
    """
    Superimpose two images with given alpha values.
    
    Note:
        If alpha is closer to 1, image1 will be more visible, and if it's closer to 0, image2 will be more visible.
    Args:
        image1 (np.ndarray): First image to apply for superimpose operation.
        image2 (np.ndarray): Second image to apply for superimpose operation.
        alpha (float): Alpha of the first image. Second image gets 1 - alpha.
            Alpha 0.5 means both images take equal part in superimpose operation. 
    Returns:
        np.ndarray: Image after superimpose operation of image1 and image2.
    """
    return cv2.addWeighted(image1, alpha, image2, 1 - alpha, 0.0)
    
def heatmap_background_subtraction(
                            videopath, 
                            video_output_path,
                            heatmap_output_path,
                            model,
                            alpha,  
                            skip_frame, 
                            aggrFrame, 
                            kernel_size, 
                            kernel_operation,
                            kernel_shape,   
                            blur_kernel_size):
    """
    This function applies the process of background subtraction to an input video. 
    The goal of this function is to compute a heatmap for a given video and save it. 
    The commented code can be uncommented if there is a need to save the video that shows the heatmap calculation process.
    
    Args:
        video_input (str): Path to the input video.
        video_output (str): Path to save the output video, if applicable.
        image_output (str): Path to save the output heatmap image.
        
        model (cv2.BackgroundSubtractor): The background subtractor model used for foreground extraction. 
        In this case, `cv2.createBackgroundSubtractorKNN()` is used with `detectShadows` set to False.    
        
        alpha (float): The transparency value for superimposing the heatmap on the frame.       
        skip_frame (int): The number of initial frames to skip before starting the background subtraction and heatmap generation process.         
        aggrFrame (int): The aggregation factor for accumulating background frames. A value of 1 means that each frame is used for accumulation, resulting in a more precise heatmap.        
        kernel_size (int, int): The size of the kernel for morphological operations. The kernel is a square-shaped matrix used for erosion and other morphological operations.        
        kernel_operation (int): The type of morphological operation to be applied. It is set to `cv2.MORPH_ERODE`, which erodes the foreground regions.
        kernel_shape (int): The shape of the kernel for morphological operations. It is set to `cv2.MORPH_ELLIPSE`, which represents an elliptical kernel.
        blur_kernel_size (tuple): The size of the kernel used to blur the frame

    Returns:
    
        dict: A dictionary containing various image representations related to the background subtraction process and the stored heatmap.
        
    """
    # Get the string names of the morphological operation and shape
    kernel_op_name = morph_name(kernel_operation)
    kernel_sh_name = morph_shape_name(kernel_shape)
    
    # Modify the final part of the output filename with the current heatmap parameters
    suffix = os.path.splitext(video_output_path)[1]
    new_suffix = f'-[A.{str(alpha)},SkFr.{str(skip_frame)},AgFr.{aggrFrame},K.{kernel_size},M.{kernel_op_name},S.{kernel_sh_name},Blur.{blur_kernel_size}]{suffix}'
    video_output_path = video_output_path.replace(suffix, new_suffix)
    heatmap_output_path = heatmap_output_path.replace(suffix, new_suffix)
                
    # Initialize background subtractor and start reading video
    background_subtractor = model
    video = load_video(videopath)
                                
    # Extract video characteristics
    height = video.height
    width = video.width
    total_frames = video.num_frames
    fps = video.frame_rate

    # Initialize video writer
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #output = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))
    
    # Initialize mask
    accumulated_image = np.zeros((height, width), np.uint8)
    
    count = 0
    with tqdm(total=total_frames) as pbar:
        for video_frame in video.get_iter():
            frame = video_frame.numpy()
            
            # Blur the frame
            frame = cv2.blur(frame, blur_kernel_size)
                       
            # Apply background subtraction
            background_filter = background_subtractor.apply(frame)
                        

            if count > skip_frame and count % aggrFrame == 0:
            
                # Apply morphological operation to the background filter
                erodated_image = apply_morph(background_filter, kernel_size=kernel_size, kernel_shape=kernel_shape, kernel_operation=kernel_operation)
                            

                # Accumulate the erodated image
                accumulated_image = add_images(accumulated_image, erodated_image)
                            

                # Normalize the accumulated image
                normalized_image = normalize_image(accumulated_image)
                            

                # Apply heatmap colors to the normalized image
                heatmap_image = apply_heatmap_colors(normalized_image)
                           

                # Superimpose the heatmap image on the frame
                frames_merged = superimpose(heatmap_image, frame, alpha)
                           

                # Show current manipulated frame - Not available in google colab ------ !
                cv2.imshow(f"Heatmap", frames_merged) 
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                # Save current manipulated frame
                #output.write(frames_merged)

                # For presentation
                if count % 100 == 0:
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-1.jpg', frame) 
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-2.jpg', background_filter)
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-3.jpg', erodated_image)
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-4.jpg', accumulated_image)
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-5.jpg', normalized_image)
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-6.jpg', heatmap_image) 
                    plt.imsave(f'outputs/Bg_Sub_Processing_Steps/{count}-7.jpg', frames_merged) 
                
            pbar.update(1)                
            count += 1

    # Save the heatmap image
    plt.imsave(heatmap_output_path.replace('.avi','.jpg'), heatmap_image, cmap='hot')

    # Release resources
    video.close()
    #output.release() 
                                
    cv2.destroyAllWindows() #------ !
                                
    return {
            'Last Frame': frame,
            'Last Merged Frame': frames_merged, 
            'Heatmap': heatmap_image,
            'Accumulated Image':accumulated_image, 
            'Normalized Image':normalized_image,
            'Erodated Image': erodated_image
           }


## Heatmap - YOLO

In this case, the heatmap is calculated using detections from YOLO (You Only Look Once), which is a real-time object detection algorithm. This algorithm identifies various object classes, including humans, within an image or video by dividing the image into a grid and predicting several bounding boxes along with their associated class probabilities within each grid cell. Therefore, only the areas where the movement of people is detected will influence the heatmap, meaning that not every movement will contribute.

I used YOLOv4-tiny to strike a good balance between performance, human detection, and model memory usage. However, any YOLO model can be used.

### Load YOLO

Load the necessary configuration, weights, and class labels required for the YOLO algorithm.

In [13]:
# Function to initialize the YOLO model
def load_yolo_model(weights_path, cfg_path, labels_path):
    """
    Initialize the YOLO model.
    
    Args:
        weights_path (str): Path to the YOLO weights file.
        cfg_path (str): Path to the YOLO configuration file.
        labels_path (str): Path to the file containing the class labels.
        
    Returns:
        net: The loaded YOLOv3 model.
        classes (list): List of class labels.
    """
    net = cv2.dnn.readNet(weights_path, cfg_path)
    with open(labels_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    return net, classes

### Heatmap with YOLO

The process_video() function utilizes detections from the YOLO algorithm to compute the heatmap of a given video.

In [14]:
# Code for presentation

def add_grid(image, grid_size):
    
    # Vertical lines
    for i in range(grid_size[0], image.shape[1], grid_size[0]):
        cv2.line(image, (i, 0), (i, image.shape[0]), (255, 0, 0), 1) 

    # Horizontal lines
    for i in range(grid_size[1], image.shape[0], grid_size[1]):
        cv2.line(image, (0, i), (image.shape[1], i), (255, 0, 0), 1)
    
    return image

def get_images_for_presentation(frame, heatmap_grid, grid_size, frame_count, width, height):
    frame = add_grid(frame, grid_size)
    plt.imsave(f'outputs/YOLO_Processing_with_Grid/{frame_count}-1.jpg', frame)
    heatmap_grid_resized = cv2.resize(heatmap_grid, (width, height), interpolation = cv2.INTER_LINEAR)
    heatmap_grid_resized = add_grid(heatmap_grid_resized, grid_size)
    plt.imsave(f'outputs/YOLO_Processing_with_Grid/{frame_count}-2.jpg', heatmap_grid_resized, cmap = 'hot')

In [15]:
def process_frame(net, classes, frame, confidence_threshold):
    """
    In this function, the image is initially prepared for input to the YOLO model. Subsequently, the detection results are retrieved. 
    For the purpose of this analysis, we are exclusively interested in detections belonging to the 'person' class and any detections with a 
    confidence level falling below a certain threshold are discarded. 
    
    For every remaining detection, the coordinates of the bounding box are extracted and returned.
    
    Args:
        net (dnn_Net): The loaded YOLO model.
        classes (list): List of class labels.
        frame (array): The frame to process.
        confidence_threshold (float): Confidence threshold for detections.
        
    Returns:
        new_detections (list): List of new detections.
    """
        
    # Resize the frame to 416x416 (size required by YOLO)
    blob = cv2.dnn.blobFromImage(frame, 1/255, (640, 480), (0, 0, 0), True, crop=False)

    # Send frame to model
    net.setInput(blob)

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Init new detections list
    new_detections = []

    # iterate over detection results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter only person detections with a confidence above the threshold
            if classes[class_id] == "person" and confidence > confidence_threshold:
                # Get the bounding box coordinates
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                new_detections.append((center_x - w // 2, center_y - h // 2, w, h))

    return new_detections


def process_video(video_path, 
                  video_output_path, 
                  heatmap_output_path, 
                  confidence_threshold, 
                  detection_frequency, 
                  grid_size,
                  model_weights, 
                  model_cfg, 
                  labels_path):

    """
    Process a video using predefined YOLO model and compute the heatmap using YOLO detections.
    
    Args:
        video_path (str): Path to the video file.
        video_output_path (str): Path for the output video file.
        heatmap_output_path (str): Path for the output heatmap image.
        model_weights (str): Path to the YOLOv3 weights file.
        model_cfg (str): Path to the YOLOv3 configuration file.
        labels_path (str): Path to the file containing the class labels.
        confidence_threshold (float): Confidence threshold for YOLO detections.
        detection_frequency (int): The frequency of detection in frames.
        grid_size (tuple): The size of the grid for the heatmap.
        
    Returns:
        list: List containing the normalized heatmap, resized heatmap grid and last frame processed.
    """

    # Load YOLO model
    net, classes = load_yolo_model(model_weights, model_cfg, labels_path)
    
    # Replace the final part of the output filename with the current heatmap parameters
    suffix = os.path.splitext(video_output_path)[1]
    new_suffix = f'-[YOLOv4-tiny,GridS.{grid_size},DectFreq.{detection_frequency},Conf.{confidence_threshold}]{suffix}'
    video_output_path = video_output_path.replace(suffix, new_suffix)
    heatmap_output_path = heatmap_output_path.replace(suffix, new_suffix)

    # Load video
    video = load_video(video_path)

    # Get frame info
    width = video.width
    height = video.height
    fps = video.frame_rate
    total_frames = video.num_frames

    # Create a video writer
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #out = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))

    # Create an empty grid for the heatmap
    heatmap_grid = np.zeros(grid_size)

    frame_count = 0
    all_coordinates = []

    with tqdm(total = total_frames) as pbar:
        for video_frame in video.get_iter():
            frame = video_frame.numpy()
            
            # Perform detection only every N frames
            if frame_count % detection_frequency == 0:
                
                detections = process_frame(net, classes, frame, confidence_threshold)
                
                for (x, y, w, h) in detections:
                    cv2.rectangle(frame, (x, y), 
                                  (x + w, y + h), (0, 255, 0), 2)

                    # Calculate the grid cells that intersect the bounding box
                    grid_x_start = max(int(x * grid_size[0] / width), 0)
                    grid_x_end = min(int((x + w) * grid_size[0] / width), grid_size[0])
                    grid_y_start = max(int(y * grid_size[1] / height), 0)
                    grid_y_end = min(int((y + h) * grid_size[1] / height), grid_size[1])

                    # Increment the count in these grid cells
                    for grid_x in range(grid_x_start, grid_x_end):
                        for grid_y in range(grid_y_start, grid_y_end):
                            heatmap_grid[grid_y, grid_x] += 1
                    
                    all_coordinates.append((x + w // 2, y + h // 2))  # Centroid

            
                # Code for images in presentation
                #get_images_for_presentation(frame, heatmap_grid, grid_size, frame_count, width, height)
                #----------------------
            
            # Not available in google colab
            cv2.imshow("Frame", frame) #------ !
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Write the frame to the output video
            #out.write(frame)
            frame_count += 1
            pbar.update(1)
    
    # Release the video and the output video
    video.close()
    #out.release()
    cv2.destroyAllWindows() #------ !
                  
    # Check if the array is empty (all zeros) before performing the division (No detections)
    if np.max(heatmap_grid) != 0:
        heatmap_normalized = heatmap_grid / (np.max(heatmap_grid))
    else:
        heatmap_normalized = heatmap_grid

    # Resize heatmap to match the video width and height
    heatmap_grid_resized = cv2.resize(heatmap_grid, (width, height), interpolation = cv2.INTER_LINEAR)
    plt.imsave(heatmap_output_path.replace('.avi','.jpg'), heatmap_grid_resized, cmap='hot')
                      
    return [heatmap_normalized, heatmap_grid_resized, frame]


## Generate heatmaps

The code below utilizes the two previously defined methods - background subtraction and YOLO detection - to generate heatmaps from videos.

### Generate Heatmap with Background Subtraction

In [16]:
# Number of videos to read for each time range
video_per_range = 5

try:
    for j, (start,end) in enumerate(time_ranges): # Iterate on time ranges

        df = splits[f'{start.time()}-{end.time()}'] # Get the df from splits dict with key equal to considered time range
        rnds = []

        for i in range(video_per_range):

            # Randomly select a video
            random_number = random.randint(0, len(df) - 1)
            while random_number in rnds:
                random_number = random.randint(0, len(df))
            rnds.append(random_number)

            video = df.loc[i] # Chosen video

            # Composing paths
            video_input = os.path.join(video['Root'], video['Filename'])
            video_output = os.path.join(MT1_VIDEO_PATH, video['Filename'])
            image_output = os.path.join(MT1_HEATMAP_PATH, video['Filename'])

            print(f'\n[Group {j+1}/{len(time_ranges)} - Range: {start.time()}-{end.time()}] Processing video {i+1}/{video_per_range}\nFrom: {video_input}\nSaving heatmap at: {image_output}\nSaving detection video at: {video_output}')

            # Calculating heatmap
            result = heatmap_background_subtraction(
                video_input,
                video_output,
                image_output,
                model = cv2.createBackgroundSubtractorKNN(detectShadows = False),
                alpha = 0.5,
                skip_frame = 50,
                aggrFrame = 1,
                kernel_size = (4,4),
                kernel_operation = cv2.MORPH_ERODE,
                kernel_shape = cv2.MORPH_ELLIPSE,
                blur_kernel_size = (3,3))

            # Show info on current processed video
            plot_images([img for key, img in result.items()],
                        subtitle = 'Current output',
                        titles = [key for key, img in result.items()],
                        images_per_row = 3)
            
except KeyboardInterrupt:
    print(f'Interrupted by user')


NameError: name 'splits' is not defined

### Generate heatmap with YOLO detection

In [17]:
video_per_range = 5
try:
    for j, (start,end) in enumerate(time_ranges): # Select time range
        print(f'\nAnalyzing {video_per_range} videos in {start.time()} - {end.time()}')
        df = splits[f'{start.time()}-{end.time()}'] # Retrieve the corresponding dataframe
        rnds = []
        for i in range(video_per_range): #range(len(df))

            # Randomly select a video from dataframe
            random_number = random.randint(0, len(df) - 1)
            while random_number in rnds:
                random_number = random.randint(0, len(df))
            rnds.append(random_number)

            video = df.loc[random_number] # Lock video info

            video_input = os.path.join(video['Root'], video['Filename'])
            video_output = os.path.join(MT2_VIDEO_PATH, video['Filename'])
            image_output = os.path.join(MT2_HEATMAP_PATH, video['Filename'])
            
            print(f'\n[Group {j+1}/{len(time_ranges)} - Range: {start.time()}-{end.time()}] Processing video {i+1}/{video_per_range}\nFrom: {video_input}\nSaving heatmap at: {image_output}\nSaving detection video at: {video_output}')

            # Generating heatmap of the extracted video
            results = process_video('dataset/2018-12-19 17-30-00~17-34-59.avi', video_output, image_output,
                          model_weights= YOLO + "/yolov4-tiny.weights",
                          model_cfg= YOLO + "/yolov4-tiny.cfg",
                          labels_path= YOLO + "/coco.names",
                          confidence_threshold=0.25,
                          detection_frequency=2,
                          grid_size = (80,80))

            # Show current output info
            plot_images(results,
                        subtitle = 'Current output',
                        titles = ['Normalized Heatmap','Resized Heatmap','Last Frame'],
                        images_per_row = 3)
except KeyboardInterrupt:
    print(f'Interrupted by user')



Analyzing 5 videos in 08:00:00 - 08:29:59


NameError: name 'splits' is not defined

# Section 3 - Show generated heatmaps

## Heatmap analysis

The following code display previously generated heatmaps and compute the occupancy.

### Presentation functions

In [18]:
def generate_color_range(lower_bound, upper_bound, name, size=256):
    colors = []

    r_range = range(lower_bound[0], upper_bound[0], 10)
    g_range = range(lower_bound[1], upper_bound[1], 10)
    b_range = range(lower_bound[2], upper_bound[2], 10)

    for r in r_range:
        for g in g_range:
            for b in b_range:
                colors.append((r, g, b))

    width, height = len(colors), size
    image_array = np.zeros((height, width, 3), dtype=np.uint8)

    colors.sort(key=sum)
    for i, color in enumerate(colors):
        image_array[:, i] = color

    # Ridimensiona l'immagine alle dimensioni desiderate
    image_resized = cv2.resize(image_array, (640, 480))
    return image_resized

def get_masks():

    # Range 1
    lower_bound1 = np.array([0, 100, 200])
    upper_bound1 = np.array([100, 190, 255])
    color_range1 = generate_color_range(lower_bound1, upper_bound1, 'Mask1')
    
    # Range 2
    lower_bound2 = np.array([0, 0, 120])
    upper_bound2 = np.array([30, 30, 200])
    color_range2 = generate_color_range(lower_bound2, upper_bound2, 'Mask2')

    # Range 3
    lower_bound3 = np.array([150, 220, 0])
    upper_bound3 = np.array([190, 255, 50])
    color_range3 = generate_color_range(lower_bound3, upper_bound3, 'Mask3')

    # Range 4 
    lower_bound4 = np.array([50, 220, 120])
    upper_bound4 = np.array([100, 255, 255])
    color_range4 = generate_color_range(lower_bound4, upper_bound4, 'Mask4')

    return [color_range1,color_range2,color_range3,color_range4]

### Heatmaps manipulation functions

In [19]:
def split_heatmaps(time_ranges, path):
    """
    Splits the heatmaps based on the given time ranges.

    Args:
        time_ranges (List[Tuple[datetime.time, datetime.time]]): List of tuples representing
            the start time and end time of each time range.
        path (str): The path to the folder containing the heatmaps.

    Returns:
        Dict[str, List[np.ndarray]]: A dictionary mapping each time range to a list of heatmaps
            that fall within that range.
    """
    heatmaps = read_images_from_folder(path)

    # Extracting the time details from the heatmaps
    heatmap_times = {
        name: (
            datetime.strptime(extract_datetime(name)[1], "%H-%M-%S").time(),
            datetime.strptime(extract_datetime(name)[2], "%H-%M-%S").time(),
        ) 
        for name in heatmaps.keys()
    }

    # Split hatmaps
    range_heatmaps = {
        f'{st.time()}-{et.time()}': {
            name : pic for name, pic in heatmaps.items()
            if heatmap_times[name][0] >= st.time() and heatmap_times[name][1] <= et.time()
        }
        for st, et in time_ranges
    }

    return range_heatmaps
    
def highlight_hot_areas(heatmap):
    """
    This function highlights the hot areas in the given heatmap image. 
    It is specifically used for the heatmap computed with the background subtraction method, applying 
    different color masks to emphasize the areas of movement.
    
    Args:
        heatmap (np.ndarray): The heatmap image.

    Returns:
        np.ndarray: The resulting image with highlighted hot areas.
    """   
    # Define the color range for movement colors
    
    # Range 1
    lower_bound1 = np.array([0, 100, 200])
    upper_bound1 = np.array([100, 190, 255])
    
    # Range 2
    lower_bound2 = np.array([0, 0, 120])
    upper_bound2 = np.array([30, 30, 200])

    # Range 3
    lower_bound3 = np.array([150, 220, 0])
    upper_bound3 = np.array([190, 255, 50])

    # Range 4 
    lower_bound4 = np.array([50, 220, 120])
    upper_bound4 = np.array([100, 255, 255])

    # Create a mask with only the pixels in bounds
    mask1 = cv2.inRange(heatmap, lower_bound1, upper_bound1)
    mask2 = cv2.inRange(heatmap, lower_bound2, upper_bound2)
    mask3 = cv2.inRange(heatmap, lower_bound3, upper_bound3)
    mask4 = cv2.inRange(heatmap, lower_bound4, upper_bound4)

    # Combine masks
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.bitwise_or(mask, mask3)
    mask = cv2.bitwise_or(mask, mask4)
    
    # Apply the mask to the image to obtain only the parts corresponding to hot colors
    hot_area = cv2.bitwise_and(heatmap, heatmap, mask=mask)

    # To transform all the evidenced areas into warm colors
    hot_area = cv2.cvtColor(cv2.cvtColor(hot_area,cv2.COLOR_RGB2HSV), cv2.COLOR_BGR2RGB)
    return hot_area, [mask1,mask2,mask3,mask4,mask]

    
def visualize_heatmap_on_background(background_path, heatmap_grid):
    """
    Overlays the heatmap on a background image and returns the resulting image.

    Args:
        background_path (str): The path to the background image.
        heatmap_grid (np.ndarray): The heatmap grid to overlay.

    Returns:
        np.ndarray: The resulting image with the heatmap overlay.
    """
    
    # Load background image
    background_img = cv2.imread(background_path)
    fig, ax = plt.subplots(figsize=(10, 10)) # Init plots

    # Overlap two the two images
    ax.imshow(background_img)
    ax.imshow(heatmap_grid, alpha=0.5)

    ax.axis('off')

    # Create a buffer to store the overlapped images
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0) # Save image in buffer
    plt.close(fig)

    # Reopen buffer and read image
    buf.seek(0)
    overlaid_img = Image.open(buf)

    return np.array(overlaid_img)
    
def show_average_heatmap(heatmap_pics, period, method, show_intermediate_results = True, threshold = 70):
        """
        This function guides the creation of a heatmap over a specific period. 
        It can displays all the individual components accordingly to the show_intermediate_results parameter.
        It show and then return the average heatmap computed for the period.
        
        Args:
            heatmaps (List[np.ndarray]): List of heatmaps.
            period (str): The time period to display the analysis for.
            method (int): The method used to compute the input heatmaps
            show_intermediate_results (bool): to print or not more details of the process.
            threshold (int): The value threshold for occupancy calculation, used only with method 1. Default is '.
    
        Returns:
            Tuple[np.ndarray, np.ndarray] or None: A tuple containing the resulting average heatmap and the heatmap overlaid on the background. 
            Returns None if no heatmaps are available for the time period.
        """
    
        if len(heatmap_pics) > 0:
            heatmaps = []
            
            if method == 0:
                masks = get_masks()
                index = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
                plt.imsave(f'outputs/Bg_Sub_Color_Masks/Mask1-{index}.png', masks[0])
                plt.imsave(f'outputs/Bg_Sub_Color_Masks/Mask2-{index}.jpg', masks[1])
                plt.imsave(f'outputs/Bg_Sub_Color_Masks/Mask3-{index}.jpg', masks[2])
                plt.imsave(f'outputs/Bg_Sub_Color_Masks/Mask4-{index}.jpg', masks[3])
                
            for i, (name, heatmap_pic) in enumerate(heatmap_pics.items()):
                            
                pic_array = np.array(heatmap_pic)  

                # Differentiate between the two methodologies
                if method == 0: # Heatmap generated with background subtraction
                    heatmap, intermediate_results = highlight_hot_areas(pic_array)
                    

                else: # Heatmap with YOLO
                    heatmap = pic_array.copy()
                    heatmap[heatmap <= threshold] = 0
                    
                heatmaps.append(heatmap)
                
                # Overlaid heatmap on background
                heatmap_on_background = visualize_heatmap_on_background(BACKGROUND_PATH, heatmap)
                
                # Occupancy estimation
                occupancy = estimate_occupancy(heatmap)
                
                # Show intermediate components
                if show_intermediate_results:
                    date, start_time, end_time = extract_datetime(name)
                    #print_title(f'Analysis for range: {period} - Heatmap {i+1} calculated from {start_time} to {end_time}')

                    if method == 0:
                        plot_images([heatmap_pic, pic_array] + masks + intermediate_results + [heatmap, heatmap_on_background],
                                    titles = ['Input Heatmap Picture','Heatmap Array','Color Range 1','Color Range 2','Color Range 3','Color Range 4','Mask 1','Mask 2','Mask 3','Mask 4','Generated Occupancy Mask (1,2,3,4)','Evidenced heatmap', 'Heatmap on background'], 
                                    subtitle = f'Heatmap {i+1} Period: {start_time}-{end_time} - Occupancy: {round(occupancy,2)}%', 
                                    images_per_row = 6)
                    else:
                        plot_images([pic_array, heatmap, heatmap_on_background], 
                                    subtitle = f'Heatmap {i+1} Period: {start_time}-{end_time} - Occupancy: {round(occupancy,2)}% - Threshold: {threshold}',
                                    titles = ['Original Heatmap',f'Heatmap with Threshold ({threshold})','Applied on Background'],
                                    images_per_row = 3)                       

                
            average_heatmap = (np.mean(heatmaps, axis=0)).astype(np.uint8)
            if method == 1:
                average_heatmap[average_heatmap <= threshold] = 0

            
            average_heatmap_on_background = visualize_heatmap_on_background(BACKGROUND_PATH, average_heatmap)
            
            occupancy = estimate_occupancy(average_heatmap)

            # Show final result
            #if method == 0:
                #print_title(f'Average Heatmap')

            #else:
                #print_title(f'Average Heatmap - Threshold {threshold}')                
            plot_images([average_heatmap, average_heatmap_on_background], 
                        subtitle=f'Average Heatmap for {period} - Average Occupancy: {round(occupancy,2)}%', 
                        titles=[f'Average Heatmap with Threshold ({threshold})', 'Average heatmap on Background'],
                        images_per_row = 2)

        else:
            print(f'No heatmap for time range available.\n')
            return None
        return average_heatmap, average_heatmap_on_background, occupancy

### Occupancy evaluation functions

The first of the following three functions calculates the occupancy of a heatmap for both methods. The second and third functions are used to display the average heatmap of a period calculated with the previous functions. This display is useful when used with heatmaps calculated using the second method (YOLO), as it allows the mouse click to evaluate the intensity of various areas of the heatmap, enabling adjustment of a parameter that indicates the value above which an area is considered occupied.

Therefore, these functions should be executed after the heatmap for a chosen time range has been composed. At that point, the calculated heatmap will be displayed, overlaid on the background.


In [20]:
def estimate_occupancy(heatmap):
    """
    Estimates the occupancy percentage. It calculate the percentage of heatmap cells that are not 0.

    Args:
        heatmap (np.ndarray): The heatmap image.

    Returns:
        float: The percentage of occupied cells in the heatmap.
    """

    active_cells = len(np.where(heatmap != 0)[0])
    inactive_cells = len(np.where(heatmap == 0)[0])
        
    # Percentage of active cells
    occupancy_percentage = (active_cells / (active_cells + inactive_cells)) * 100
    
    return occupancy_percentage

def show_intensity(event, x, y, flags, param):
    """
    Handles the mouse click event to show the average intensity of a grid cell.
    
    Args:
        event (cv2.EVENT): The OpenCV mouse event.
        x (int): The x-coordinate of the mouse event.
        y (int): The y-coordinate of the mouse event.
        flags (int): Any relevant flags for the event.
        param (tuple): Additional parameters. Expects a tuple with the original image, 
                       the heatmap, and the grid size (height, width).
    """
    # Mouse click event
    if event == cv2.EVENT_LBUTTONUP:
        image, heatmap, method = param

        # Ensure pixel indices are within valid limits
        x = min(x, image.shape[1] - 1)
        y = min(y, image.shape[0] - 1)

        # Calculate intensity at the pixel
        pixel_intensity = np.mean(heatmap[y, x])
        if pixel_intensity < pixel_intensity and method == 1:
            pixel_intensity = 0
        print(f"Intensity at pixel ({x}, {y}): {pixel_intensity}")

def show_intensity_on_image(image, heatmap, method):
    """
    Visualizes the grid overlay on an image and sets up the mouse callback to show 
    the intensity when a pixel is clicked. The intensity is retrieved from the heatmap.
    The image showed is the average heatmap overlapped on background.

    Args:
        image (np.ndarray): The original image.
        heatmap (np.ndarray): The heatmap.
    """
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    heatmap = np.array(heatmap)

    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    
    # Register the mouse callback function
    cv2.namedWindow("Inspect itensity")
    cv2.setMouseCallback("Inspect itensity", show_intensity, (image, heatmap, method))

    # Show the image
    cv2.imshow("Inspect itensity", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Heatmap Visualization

### Visualize heatmaps generated with background subtraction

In [21]:
# Bring in memory available heatmaps calculated with background subtraction
heatmaps_splitted = split_heatmaps(time_ranges, MT1_HEATMAP_PATH)
available_choices = list([key for key, list in heatmaps_splitted.items() if len(list) > 0])

choice = make_choice(available_choices, title = f'Select time range:\n')

if choice != 0:
    index = available_choices[choice - 1]
    heatmap, heatmap_on_background, occupancy = show_average_heatmap(heatmap_pics = heatmaps_splitted[index], 
                                                       period = index, 
                                                       method = 0, 
                                                       show_intermediate_results = True)
    index = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    plt.imsave(f'outputs/Bg_Sub_Average_Heatmap/Heatmap{index}.jpg', heatmap)
    plt.imsave(f'outputs/Bg_Sub_Average_Heatmap/HeatmapOnBackground-{index}.jpg', heatmap_on_background)

Select time range:

0 - Exit


ValueError: invalid literal for int() with base 10: ''

### Dinamically detect heatmap intensity

Analyze intensity on previously generated heatmap.

In [ ]:
show_intensity_on_image(heatmap_on_background, heatmap, 0)

### Visualise heatmaps generated with YOLO detection

In [23]:
"""
Note for M2_HEATMAP_THRESHOLD

Occupancy level for occupancy percentage calculation.
By increasing this value, only areas with a value above the threshold will contribute to the heatmap calculation.
Can be used to smooth the heatmap intensity and narrow the area of interest to the hottest ones.
For example, with 0 it use all shades of the heatmap in calculating masks and occupancy.
"""
M2_HEATMAP_THRESHOLD = 40

heatmaps_splitted = split_heatmaps(time_ranges, MT2_HEATMAP_PATH) # Bring in memory available heatmap calculated with background subtraction
available_choices = list([key for key, list in heatmaps_splitted.items() if len(list) > 0])

choice = make_choice(available_choices, title = f'Select time range:\n')

if choice != 0:
    index = available_choices[choice - 1]
    heatmap, heatmap_on_background, occupancy = show_average_heatmap(heatmap_pics = heatmaps_splitted[index], 
                                                           period = index, 
                                                           method = 1, 
                                                           show_intermediate_results = False,
                                                           threshold = M2_HEATMAP_THRESHOLD)
    
    index = index = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    plt.imsave(f'outputs/YOLO_Average_Heatmap/HeatmapWithThreshold-{index}.jpg', heatmap)
    plt.imsave(f'outputs/YOLO_Average_Heatmap/HeatmapOnBackground-{index}.jpg', heatmap_on_background)

Select time range:

0 - Exit
Exit.


### Dinamically detect heatmap intensity

Analyze intensity on previously generated heatmap.

In [ ]:
show_intensity_on_image(heatmap_on_background, heatmap, 1)